# ENSO_seasonality: Seasonal Variability of SST Anomalies in the Central Equatorial Pacific  

Computes the ratio of boreal winter (November–January, NDJ) to spring (March–May, MAM) standard deviation of sea surface temperature anomalies (SSTA) in the Niño3.4 region (horizontal average over 5°S–5°N, 170°W–120°W). This metric evaluates the seasonality of ENSO variability, characterised by maximum variability in winter and minimum variability in spring in the observations.  

## Datasets Used for Seasonality Comparison  

The following reference datasets are used to evaluate the seasonal variability of ENSO-related SST anomalies in model simulations:  
- **ERSSTv5 (1854–2018)**: A globally gridded dataset optimised for climate monitoring, including ENSO-related SST anomaly studies.  

## Exercises
In this notebook there are some cells to be filled out to complete with ESM1.6 CMORised data. This is up to diagnostic level 3 from which you can modify the rest to include as you desire.

## Dask Cluster 

Start a cluster with multiple cores 

In [ ]:
from dask.distributed import Client, progress
client = Client(threads_per_worker = 1)
client

## Comparison for ESM1.6

- Use ACCESS-MOPPeR interactive API to CMORise required variable. (May take some time)
- Write output to netCDF to read in later in other notebooks too
- or convert to iris cube to tie into recipe
- **ACCESS-MOPPeR will require at least xxlarge (28cpus, 126G mem)**

In [ ]:
import glob
from access_mopper import ACCESS_ESM_CMORiser

ROOT_FOLDER = (
    "/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/"
)

OUTPUT_FOLDER = "/scratch/nf33/$USER"

In [ ]:
parent_experiment_config = {
    "parent_experiment_id": "piControl",
    "parent_activity_id": "CMIP",
    "parent_source_id": "ACCESS-ESM1-5",
    "parent_variant_label": "r1i1p1f1",
    "parent_time_units": "days since 0001-01-01 00:00:00",
    "parent_mip_era": "CMIP6",
    "branch_time_in_child": 0.0,
    "branch_time_in_parent": 54786.0,
    "branch_method": "standard",
}

In [ ]:
FILES = glob.glob(ROOT_FOLDER + "output0[0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)
## reduce the years to cmorise by the output folders

In [ ]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.ts", #
    experiment_id="piControl", #
    source_id="ACCESS-ESM1-5", # metadata
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config,  # <-- This is optional, can be skipped if not needed
)

In [ ]:
cmoriser.run()

In [ ]:
# cmoriser.write()
esm16 = cmoriser.to_iris()

### If not runnng MOPPeR or do not have native outputs
There is some cmorised spin up data you can use located in `/scratch/nf33/MOPPeR_ENSO/`

In [ ]:
!ls /scratch/nf33/MOPPeR_ENSO

In [ ]:
## if you have other cmorised file already saved out example
fp = '/scratch/nf33/MOPPeR_ENSO/ts_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc'
import iris
iris.load_cube(fp)
# esm16 = iris.load_cube(fp)

In [ ]:
# check: this is a cubelist of one cube
esm16

## Datasets

In [ ]:
from esmvalcore.dataset import Dataset

In [ ]:
model_datasets = {
"ACCESS-ESM1-5": 
    Dataset(
    short_name='ts',
    project='CMIP6',
    mip="Amon",
    exp="historical",
    ensemble="r1i1p1f1",
    timerange="18500101/20190101",
    dataset="ACCESS-ESM1-5",
    grid="gn"
),
}

obs_datasets = {
"ERSSTv5":
    Dataset(
    short_name='tos',
    dataset='NOAA-ERSSTv5',
    mip="Omon",
    project='OBS6',
    type='reanaly',
    tier=2),
}

## Add Ancillary files

In [ ]:
for name, dataset in model_datasets.items():
    dataset.add_supplementary(short_name='sftlf', mip="fx")
    dataset.add_supplementary(short_name="areacella", mip="fx")

In [ ]:
model_datasets = {name: dataset.load() for name, dataset in model_datasets.items()}
obs_datasets = {name: dataset.load() for name, dataset in obs_datasets.items()}

In [ ]:
from esmvalcore.preprocessor import anomalies
from esmvalcore.preprocessor import area_statistics
from esmvalcore.preprocessor import climate_statistics
from esmvalcore.preprocessor import convert_units
from esmvalcore.preprocessor import extract_region
from esmvalcore.preprocessor import extract_time
from esmvalcore.preprocessor import extract_season
from esmvalcore.preprocessor import regrid
from esmvalcore.preprocessor import detrend
from esmvalcore.preprocessor import meridional_statistics
from esmvalcore.preprocessor import mask_landsea
import iris

import numpy as np

import matplotlib.pyplot as plt
import iris.quickplot as qplt

In [ ]:
def sst_boreal(cube, season): #season->'NDJ','MAM'
    nino34_latext_region = {"start_longitude": 190., "end_longitude": 240., "start_latitude": -5., "end_latitude": 5.}
    # cube = extract_time(cube, start_year=1900, start_month=1, start_day=1, end_year=2014, end_month=12, end_day=31)
    cube = mask_landsea(cube, mask_out="land")
    cube = regrid(cube, target_grid="1x1", scheme="linear")
    cube = convert_units(cube, units="degC")
    cube = extract_region(cube, **nino34_latext_region)
    cube = anomalies(cube,period='monthly')
    cube = area_statistics(cube,operator='mean')
    cube = extract_season(cube, season) # get NDJ
    cube = climate_statistics(cube, operator="std_dev", period="full")
    return cube


In [ ]:
def sst_std(cube):
    nino34_latext_region = {"start_longitude": 190., "end_longitude": 240., "start_latitude": -5., "end_latitude": 5.}
    # cube = extract_time(cube, start_year=1900, start_month=1, start_day=1, end_year=2014, end_month=12, end_day=31)
    cube = mask_landsea(cube, mask_out="land")
    cube = regrid(cube, target_grid="1x1", scheme="linear")
    cube = extract_region(cube, **nino34_latext_region)
    cube = anomalies(cube,period='monthly')
    cube = area_statistics(cube, operator='mean')
    cube = climate_statistics(cube, operator="std_dev", period="monthly") #monthly to plot months
    return cube
    
def sst_std_3(cube):
    nino34_latext_region = {"start_longitude": 130., "end_longitude": 280., "start_latitude": -5., "end_latitude": 5.}
    # cube = extract_time(cube, start_year=1900, start_month=1, start_day=1, end_year=2014, end_month=12, end_day=31)
    cube = mask_landsea(cube, mask_out="land")
    cube = regrid(cube, target_grid="0.5x0.5", scheme="linear")
    cube = extract_region(cube, **nino34_latext_region)
    cube = anomalies(cube,period='monthly')
    cube = meridional_statistics(cube, 'mean')
    cube = climate_statistics(cube, operator="std_dev", period="monthly")
    return cube

Note: some warnings will appear due the processes such as extract and regiona and regriding will have some 'Nan' or invalid values

In [ ]:
model_datasets_prep = {name: sst_boreal(dataset, 'NDJ') for name, dataset in model_datasets.items()}
obs_datasets_prep = {name: sst_boreal(dataset,'NDJ') for name, dataset in obs_datasets.items()}


In [ ]:
model_datasets_prep2 = {name: sst_boreal(dataset, 'MAM') for name, dataset in model_datasets.items()}
obs_datasets_prep2 = {name: sst_boreal(dataset,'MAM') for name, dataset in obs_datasets.items()}

### exercise: prepare ESM1.6 data

In [ ]:
esm16_prep = sst_boreal(,'NDJ') #esm16[0]

In [ ]:
# computes the data
esm16_prep.data.mean()

In [ ]:
esm16_prep2 = sst_boreal(,'MAM') #esm16[0]

## Diagnostic Level 1

The first level shows the diagnostic used to compute the metric and highlight the difference between the model and the reference.

In [ ]:
model_ = {'borealwinter':model_datasets_prep["ACCESS-ESM1-5"], 'borealspring':model_datasets_prep2["ACCESS-ESM1-5"]}

obs_ = {'borealwinter':obs_datasets_prep["ERSSTv5"], 'borealspring':obs_datasets_prep2["ERSSTv5"]}

mod_seas = model_['borealwinter'].data.item()/model_['borealspring'].data.item()
obs_seas = obs_['borealwinter'].data.item()/obs_['borealspring'].data.item()

In [ ]:
esm16_seas = esm16_prep.data / esm16_prep2.data

In [ ]:
ssta_std = [mod_seas, obs_seas, esm16_seas] ## add to plot
plt.scatter(range(len(ssta_std)), ssta_std, c=['blue','black', 'tab:blue'], marker='D')

plt.xlim(-0.5,3)#range(-1,3,1)) #['model','obs']
plt.xticks([])

plt.ylabel('SSTA std (NDJ/MAM) (°C/°C)')
plt.grid(linestyle='--', axis='y')
plt.title('ENSO Seasonality')

metricval = abs((mod_seas-obs_seas)/obs_seas)*100

plt.text(0.95, 0.95, f'Metric: {metricval:.2f} %', fontsize=12, ha='right', transform=plt.gca().transAxes,
         bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

plt.text(0.75,0.85, '* ACCESS-ESM1-5', color='blue',transform=plt.gca().transAxes)
plt.text(0.75,0.8, '* observation', color='black',transform=plt.gca().transAxes)


**Figure 1**: Ratio of winter to spring standard deviation of sea surface temperature anomalies (SSTA) in the central equatorial Pacific (Niño 3.4 region average), illustrating the seasonal timing of SSTA. The model generally exhibits weaker variability compared to observations, indicating that ENSO events tend to occur too frequently during spring in the model. In this case, the seasonal timing is slightly overrepresented. Black markers denote the reference data, while blue markers denote the model output. 


## Diagnostic Level 2

The second level shows the mean annual structure of the ENSO amplitude: the 12 months standard deviation of the anomalies.

In [ ]:
model_data_prep = {name: sst_std(dataset) for name, dataset in model_datasets.items()}
obs_data_prep = {name: sst_std(dataset) for name, dataset in obs_datasets.items()}

data_prep_2 = [model_data_prep["ACCESS-ESM1-5"], obs_data_prep["ERSSTv5"]]

In [ ]:
esm16_prep_std = sst_std(esm16[0])

In [ ]:
# Create the plot
plt.figure(figsize=(6, 6))

# Plot observation data in black
qplt.plot(data_prep_2[1], color='black', label='Observation', linewidth=4)

# Plot model data in blue
qplt.plot(data_prep_2[0], color='blue', label='ACCESS-ESM1-5', linewidth=4)
# Plot spin up
qplt.plot(esm16_prep_std)

# Define x-axis labels
# months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months = ['Jan', 'May', 'Sep']
plt.xticks(range(1,13,4),labels=months)
# Set the x and y axis labels
plt.xlabel('Months')
plt.ylabel('SSTA std (°C)')
plt.title('SSTA standard deviation')
plt.grid(linestyle='--'); plt.legend()


**Figure 2**: Mean annual structure of the standard deviation of sea surface temperature anomalies (SSTA) in the central equatorial Pacific (Niño 3.4 region average), illustrating the seasonal variability. The model typically shows too strong variability in winter and in spring. The black curve represents the reference data, while the blue curve corresponds to the model output.

## Diagnostic Level 3

The third level shows the spatio-mean annual structure of the ENSO amplitude: the Hovmöller of the standard deviation of anomalies in the equatorial Pacific.

In [ ]:
model_data_prep3 = {name: sst_std_3(dataset) for name, dataset in model_datasets.items()}
obs_data_prep3 = {name: sst_std_3(dataset) for name, dataset in obs_datasets.items()}

data_prep_3 = {"ERSSTv5":obs_data_prep3["ERSSTv5"], **model_data_prep3} #for plot

In [ ]:
import iris.plot as iplt
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

# Function to format longitude labels
def format_longitude(x, pos):
    if x > 180:
        return f'{int(360 - x)}°W'
    elif x == 180:
        return f'{int(x)}°'
    else:
        return f'{int(x)}°E'

In [ ]:
# Set up the figure
fig = plt.figure(figsize=(15, 7))
i = 121  # Subplot indexing

# Define month labels for the y-axis
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Plot data for each label and cube in the dictionary
for label, cube in data_prep_3.items():
    ax = plt.subplot(i)  # Add projection for geographic plots
    c1 = iplt.contourf(cube, cmap='Reds', levels=np.arange(0., 2.1, 0.1))  # Set levels to include <0 and >2
    ax.set_yticks(range(1, 13))
    ax.set_yticklabels(months)
    ax.set_title(label)
    ax.set_ylabel("Months")
    ax.set_xlabel("Longitude")
    ax.xaxis.set_major_formatter(plt.FuncFormatter(format_longitude))
    i += 1

fig.subplots_adjust(bottom=0.15, right=1.0)  # Adjust bottom margin to fit colorbar
# Add a horizontal colorbar with open ends to include values outside the range
cax = plt.axes([0.15, -0.08, 0.7, 0.05])  # Position for colorbar
cbar = fig.colorbar(c1, cax=cax, orientation='horizontal', extend='both', ticks=np.arange(0, 2.2, 0.5))
cbar.set_label('SSTA std (°C)')

plt.show()

Figure 3: Spatio-mean annual structure of the standard deviation of sea surface temperature anomalies (SSTA) in the equatorial Pacific (averaged between 5°S and 5°N). The figure typically shows too weak variability off the coast of South America and too strong variability in the central Pacific during spring. Additionally, there is too weak variability in the central-eastern Pacific during winter and excessive variability west of the dateline year-round. In this case, while the seasonal timing of ENSO is roughly captured, the variability in the central Pacific is overly strong during autumn. The left map displays the reference data, while the right map shows the model output.

### Dive down level 4:
The fourth level shows the zonal structure of the ENSO amplitude during winter and spring: the standard deviation of the anomalies along the equator in the Pacific.

In [ ]:
## from level 3 -extract season, full climate statictics
def sst_std_4(cube):
    nino34_latext_region = {"start_longitude": 150., "end_longitude": 270., "start_latitude": -5., "end_latitude": 5.}
    cube = regrid(cube, target_grid="1x1", scheme="linear")
    cube = mask_landsea(cube, mask_out="land")
    cube = convert_units(cube, units="degC")
    cube = extract_region(cube, **nino34_latext_region)
    cube = anomalies(cube,period='monthly')
    cube = meridional_statistics(cube, 'mean')

    return cube

model_data_prep4 = {name: sst_std_4(dataset) for name, dataset in model_datasets.items()}
obs_data_prep4 = {name: sst_std_4(dataset) for name, dataset in obs_datasets.items()}
data_prep_4 = {**model_data_prep4, "ERSSTv5":obs_data_prep4["ERSSTv5"]}

In [ ]:
## run for esm16[0] and add for plotting


In [ ]:
        
lines = {'ACCESS-ESM1-5': 'solid', 'ERSSTv5': 'dashdot'}
for label, cube in data_prep_4.items():
    for seas, col in {'NDJ':'red','MAM':'blue'}.items():
        cube_plot = extract_season(cube, seas) # get NDJ #{ NDJ, MAM, red, blue
        cube_plot = climate_statistics(cube_plot, operator="std_dev", period="full")
        qplt.plot(cube_plot, color=col, label=label + " " + seas,linestyle=lines[label], linewidth=2)
        
##
plt.xlabel('longitude')
plt.ylabel('SSTA std (°C)')
plt.title('SSTA standard deviation')
plt.legend()
plt.grid(linestyle='--')
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(format_longitude))

**Figure 4**: Zonal structure of the standard deviation of sea surface temperature anomalies (SSTA) in the equatorial Pacific (averaged between 5°S and 5°N) during winter (red curves) and spring (blue curves). In winter, the model typically shows weaker variability in the central-eastern Pacific and stronger variability west of the dateline. In spring, the model shows weaker variability off the coast of South America and stronger variability elsewhere. Although the zonal structure of the ENSO amplitude is captured reasonably well, the variability off South America is too weak in both seasons. The dashed curves represent the reference data, while the solid curves represent the model output.

## Dive down level 5:

In [ ]:
def sst_std_5(cube):
    nino34_latext_region = {"start_longitude": 120., "end_longitude": 290., "start_latitude": -15., "end_latitude": 15.}
    cube = mask_landsea(cube, mask_out="land")
    cube = regrid(cube, target_grid="1x1", scheme="linear")
    cube = convert_units(cube, units="degC")
    cube = extract_region(cube, **nino34_latext_region)
    cube = anomalies(cube, period='monthly')

    return cube

model_data_prep5 = {name: sst_std_5(dataset) for name, dataset in model_datasets.items()}
obs_data_prep5 = {name: sst_std_5(dataset) for name, dataset in obs_datasets.items()}
data_prep_5 = {"HadISST":obs_data_prep5["HadISST"], **model_data_prep5}
process = {}


for seas in ['NDJ','MAM']:
    for label, cube in data_prep_5.items():
        cube_plot = extract_season(cube, seas)
        cube_plot = climate_statistics(cube_plot, operator="std_dev", period="full")
        process[f'{label} {seas}'] = cube_plot


In [ ]:
fig = plt.figure(figsize=(20, 5))
proj = ccrs.PlateCarree(central_longitude=180)

i =221
for label, cube in process.items(): 
    
    ax1 = plt.subplot(i,projection=proj)
    ax1.coastlines()
    cf1 = iplt.contourf(cube, levels=np.arange(0.,2,0.1), extend='both', cmap='Reds')
    ax1.set_title(label)
    gl1 = ax1.gridlines(draw_labels=True, linestyle='--')
    gl1.top_labels = False
    gl1.right_labels = False
    
    i+=1
    
cax = plt.axes([0.15,0.01,0.7,0.03])
cbar = fig.colorbar(cf1, cax=cax, orientation='horizontal', extend='both', ticks=np.arange(0.,2.3,0.5))
cbar.set_label('monthly SSTA std (°C)')

Figure 5: spatial structure of the standard deviation of sea surface temperature anomalies (SSTA) in the equatorial Pacific, showing usually a too weak variability off South America and too strong variability west of the dateline during both seasons, a too strong (weak) variability in the central equatorial Pacific during spring (winter) (here the variability is also too weak off South America during both seasons and the variability is too strong but not spread enough in latitude in the central Pacific during winter). The left and right maps show respectively the reference and the model. The first and second rows show respectively boreal winter and spring.

In [ ]:
client.close()